### Views generator with randomized generated data

In [15]:
from kafka import KafkaProducer

import json
import requests
from datetime import datetime
import time

In [16]:
producer = KafkaProducer(
    bootstrap_servers=['ec2-3-95-18-154.compute-1.amazonaws.com:9092'], #here input the correct DNS of our kafka server
)

The following code generates data based on the introduced posts

In [17]:
from numpy.random import choice, randint, normal

In [18]:
API_URL = "http://localhost:5000"
API_GET_COMMENTS = f"{API_URL}/comments"

def get_comments():
    
    response = requests.get(API_GET_COMMENTS)
    body = response.json()
    
    return body

In [19]:
comments = get_comments()
comment_ids = [c["_id"] for c in comments]
comment_ids

['43e28d99-4a1c-47f1-b754-9b9e5a02e2ea',
 '9655bbb4-18a3-4642-8c58-7d945ae68244',
 '81e5c19b-0302-4b33-8aa4-31b3221586b4',
 '29df8ed8-5d44-4a34-9462-58fdd80cf046',
 '5af40970-de0e-4e52-8125-b9c6b4c4f971',
 '4456b5c6-bbd4-404b-acf1-0d7f891c6f54',
 '68bde94d-87b7-440f-ad49-e82f26f47d38',
 '87129086-9aa5-49f1-955b-77b080a0598c',
 '1c23c981-c765-44fd-83c5-b89839927ff3',
 'ae87fd20-80ca-46e1-af5e-7faa4fe21692',
 '93fad7f0-1ad9-477a-a67f-367b564f1b0e',
 '5a6ec4ea-e431-49c5-90c2-27e61ed9c64d',
 'b2d1923c-98a9-4166-b322-e307385b99a3',
 'efeba57c-d647-4486-96c0-c2b74a545e89']

We give a random weight to the comments

In [20]:
comment_weights = randint(1, 10, len(comments))
comment_weights = comment_weights / comment_weights.sum()

comment_weights

array([0.07692308, 0.07692308, 0.08974359, 0.08974359, 0.03846154,
       0.03846154, 0.05128205, 0.11538462, 0.07692308, 0.11538462,
       0.1025641 , 0.02564103, 0.07692308, 0.02564103])

We generate random data for few aditional variables

In [21]:
users = [f'user{i}' for i in range(100)]

devices = ['mobile', 'desktop', 'other']
devices_weights = [0.6, 0.3, 0.1]

geos = ["A Coruña", "Álava", "Albacete", "Alicante", "Almería", "Asturias", "Ávila", "Badajoz", "Islas Baleares", "Barcelona", "Burgos", "Cáceres", "Cádiz", "Cantabria", "Castellón", "Ciudad Real", "Córdoba", "Cuenca", "Girona", "Granada", "Guadalajara", "Guipúzcoa", "Huelva", "Huesca", "Jaén", "La Rioja", "Las Palmas", "León", "Lleida", "Lugo", "Madrid", "Málaga", "Murcia", "Navarra", "Orense", "Palencia", "Pontevedra", "Salamanca", "Segovia", "Sevilla", "Soria", "Tarragona", "Santa Cruz de Tenerife", "Teruel", "Toledo", "Valencia", "Valladolid", "Vizcaya", "Zamora", "Zaragoza"]
geos_weights = randint(1, 10, len(geos))
geos_weights = geos_weights / geos_weights.sum()


In [22]:
def gen_fake_data():
    return {
        'comment_id': choice(comment_ids, p=comment_weights),
        'user': choice(users),
        'timestamp': str(datetime.now()),
        'device': choice(devices, p=devices_weights),
        'geo': choice(geos, p=geos_weights),
        'minutes': max(normal(2.5, 2), 0.0)
    }

In [23]:
gen_fake_data()

{'comment_id': '9655bbb4-18a3-4642-8c58-7d945ae68244',
 'user': 'user51',
 'timestamp': '2022-03-18 18:29:24.447376',
 'device': 'desktop',
 'geo': 'Vizcaya',
 'minutes': 0.9891024385446667}

The next code is a loop that continously sends data to Kafka

In [24]:
def put_to_stream(payload):
    producer.send('views', json.dumps(payload).encode())

In [25]:
def start_stream():
    while True:
        payload = gen_fake_data()
        put_to_stream(payload)
        time.sleep(max(normal(0.5, 0.1), 0))


In [26]:
start_stream()

KeyboardInterrupt: 